<a href="https://colab.research.google.com/github/moh2236945/pytorch_classification/blob/master/models/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Basic Library
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_EPOCHS=90
Batch_size=128
MOMENTUM=0.9
LR_DECCAY=0.005
LR_INIT=0.01
IMAGE_DIM=227
NUM_ClASSES=1000
DEVICE_IDS=[0,1,2,3]
INPUT_ROOT_DIR='data_in'
OUTPUT_DIR='alexnet_data_out'
LOG_DIR=OUTPUT_DIR+'/logs'
CHECKPOINT_DIR=OUTPUT_DIR+'/models'
#checkpoint path directory
os.makedirs(CHECKPOINT_DIR,exist_ok=True)


In [ ]:
class AlexNet(nn.Module):
    def __init(self,num_classes=1000):
        super().__init__()
        self.net=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=96,kerenel_size=11,stride=4),
            nn.Relu(),
            nn.LocalResponsenorm(size=5,alpha=0.0001,beta=0.75,k=2),
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.Conv2d(96,256,5,padding=2),
            nn.Relu(),
            nn.localResponsenorm(size=5,aplpha=0.0001,beta=0.75,k=2),
            nn.Maxpool2d(kernel_size=3,stride=2),
            nn.Conv2d(256,384,3,padding=1),
            nn.Relu(),
            nn.Conv2d(384,256,3,padding=1),
            nn.Relu()
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier=nn.Sequential(
            nn.Dropout(p=0.5,inplace=True),
            nn.Linear(in_features=(256*6*6),out_features=4096),
            nn.Relu(),
            nn.Dropout(p=0.5,inplace=True),
            nn.Linear(in_features=4096,out_features=4096),
            nn.Relu(),
            nn.Linear(in_features=4096,out_features=num_classes),
        )
        self.init_bias()# initialize the bias
    def init_bias(self):
        for layers in self.net:
            if isinstance(layer,nn.Conv2d):
                nn.init.normal_(layer.weight,mean=0,std=0.01)
                nn.init.constant_(layer.bias,0)
                # original paper = 1 for Conv2d layers 2nd, 4th, and 5th conv layers
        nn.init.constant_(self.net[4].bias, 1)
        nn.init.constant_(self.net[10].bias, 1)
        nn.init.constant_(self.net[12].bias, 1)

        def forward(self,x):
            x=self.net(x)
            x=x.view(-1,256*6*6)
            return self.classifier(x)

if __name__=='__main__':
    alexnet=AlexNet(num_classes=NUM_ClASSES).to(device)
    alexnet=torch.nn.parallel.DataParallel(alex_net,device_ids==DEVICE_IDS)
    print(alexnet)
    dataset=datasets.ImageFolder(TRAIN_IMG_DIR,transform.Compose([transform.CenterCrop(IMAGE_DIM),
                                                                  transform.ToTensor,
                                                                  transform.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),]))
    dataloader=data.dataLoader(dataset,shuffle=true,num_workers=8,
                               drop_last=True,batxh_size=BATCH_SIZE)
    #create optimzer
    optimizer=optim.Adam(params=alexnet.parameters(),Ir=0.0001)
    Ir_schedular=optim.Ir_scheduler.StepLR(optimizer,step_size=30,gamma=0.1)

    for epoch in range(NUM_EPOCHES):
        Ir_scheduler.step()
        for imgs,classes in dataloader:
            imgs,classes=img.to(device),classes.to(device)
            output=alexnet(imgs)
            loss=F.cross_entropy(output,classes)
            #u[date optimizer
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()]
            # log the information and add to tensorboard
            if total_steps % 10 == 0:
                with torch.no_grad():
                    _, preds = torch.max(output, 1)
                    accuracy = torch.sum(preds == classes)

                    print('Epoch: {} \tStep: {} \tLoss: {:.4f} \tAcc: {}'
                        .format(epoch + 1, total_steps, loss.item(), accuracy.item()))
                    tbwriter.add_scalar('loss', loss.item(), total_steps)
                    tbwriter.add_scalar('accuracy', accuracy.item(), total_steps)

            # print out gradient values and parameter average values
            if total_steps % 100 == 0:
                with torch.no_grad():
                    # print and save the grad of the parameters
                    # also print and save parameter values
                    print('*' * 10)
                    for name, parameter in alexnet.named_parameters():
                        if parameter.grad is not None:
                            avg_grad = torch.mean(parameter.grad)
                            print('\t{} - grad_avg: {}'.format(name, avg_grad))
                            tbwriter.add_scalar('grad_avg/{}'.format(name), avg_grad.item(), total_steps)
                            tbwriter.add_histogram('grad/{}'.format(name),
                                    parameter.grad.cpu().numpy(), total_steps)
                        if parameter.data is not None:
                            avg_weight = torch.mean(parameter.data)
                            print('\t{} - param_avg: {}'.format(name, avg_weight))
                            tbwriter.add_histogram('weight/{}'.format(name),
                                    parameter.data.cpu().numpy(), total_steps)
                            tbwriter.add_scalar('weight_avg/{}'.format(name), avg_weight.item(), total_steps)

            total_steps += 1

        # save checkpoints
        checkpoint_path = os.path.join(CHECKPOINT_DIR, 'alexnet_states_e{}.pkl'.format(epoch + 1))
        state = {
            'epoch': epoch,
            'total_steps': total_steps,
            'optimizer': optimizer.state_dict(),
            'model': alexnet.state_dict(),
            'seed': seed,
        }
        torch.save(state, checkpoint_path)


SyntaxError: ignored